# Hive assignment 1

The purpose of this task is to create an external table on the posts data of the `stackoverflow.com` website.

## Step 1. Intro. Creation of the DB

Let's create the sandbox database where you will complete your assignment.

<b>Note!</b> This code shouldn't be in your submission. Please, remove this code from the notebook before submission.

In [3]:
%%writefile creation_db.hql
DROP DATABASE IF EXISTS demodb CASCADE;

Overwriting creation_db.hql


In [4]:
%%writefile -a creation_db.hql
CREATE DATABASE demodb LOCATION '/user/jovyan/test_metastore';

Appending to creation_db.hql


In [5]:
! hive -f creation_db.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Exception in thread "main" java.lang.RuntimeException: org.apache.hadoop.hdfs.server.namenode.SafeModeException: Cannot create directory /tmp/mydir/jovyan/c1bc6316-9264-452e-bcbd-1371ec7a8fe7. Name node is in safe mode.
The reported blocks 931 needs additional 2 blocks to reach the threshold 0.9990 of total blocks 934.
The number of live datanodes 1 has reached the minimum number 0. Safe mode will be turned off automatically once the thresholds have been reached.
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.newSafemodeException(FSNamesystem.java:1372)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkNameNodeSafeMode(FSNamesystem.java:1360)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.mkdirs(FSNamesystem.java:2969)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.mkdirs(NameNodeRpcServer.java:1078)
	at org.ap

**Don't forget to remove this code before submission!**


## Step 2. Exploration of the dataset

Okay, we have created the database. Let's create your own table for users and posts.

First of all, let's watch at the datasets for `users` which are located at `/data/stackexchange1000/posts` and for `posts` which is located at `/data/stackexchange1000/users`. Print the first three rows of those datasets.


In [4]:
# Your code here. Print the first three rows of posts
!hdfs dfs -cat /data/stackexchange1000/posts/part-00000 | head -n 3

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
reporter:status:Reading 	
<row Id="1394" PostTypeId="2" ParentId="1390" CreationDate="2008-08-04T16:38:03.667" Score="16" Body="&lt;p&gt;Not sure how credible &lt;a href=&quot;http://www.builderau.com.au/program/windows/soa/Getting-started-with-Windows-Server-2008-Core-edition/0,339024644,339288700,00.htm&quot; rel=&quot;nofollow noreferrer&quot;&gt;this source is&lt;/a&gt;, but:&lt;/p&gt;&#xA;&#xA;&lt;blockquote&gt;&#xA;  &lt;p&gt;The Windows Server 2008 Core edition can:&lt;/p&gt;&#xA;  &#xA;  &lt;ul&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the file server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the Hyper-V virtualization server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the Directory Services role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the DHCP

In [5]:
# Your code here. Print the first three rows of users
!hdfs dfs -cat /data/stackexchange1000/users/part-00000 | head -n 3

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
reporter:status:Reading 	
<row Id="756" Reputation="2358" CreationDate="2008-08-08T15:31:50.013" DisplayName="Simon Gillbee" LastAccessDate="2016-12-09T15:38:03.453" WebsiteUrl="http://simon.gillbee.com" Location="Pearland, TX" AboutMe="&lt;p&gt;Personally, I am a husband, step-father, grandfather, Christ-worshiper, singer, worship leader, computer programmer, reader, game player, kite flyer, generally all around fun guy (or that fungi?).&lt;/p&gt;&#xA;&#xA;&lt;p&gt;Professionally, I've been developing both commercial and proprietary systems for the last 20 years. These days I'm primarily writing enterprise-scale client and server software using .NET and loving it!&lt;/p&gt;&#xA;&#xA;&lt;h1&gt;SOreadytohelp&lt;/h1&gt;&#xA;" Views="478" UpVotes="352" DownVotes="25" Age="45" Accoun

As you can see, those rows contain some information about posts and users in XML format.

<b>Question.</b> Which fields for users and posts do you think are the most important for the analysis? And for joining tables? 
OwnerUserId from Posts
Id from Users

<h3><b>Please, check your answer with this information!</b></h3>

So, the lines not started with the "row" tags should be ignored. The valid row contains the following fields and their order is not defined:

* Id (integer) - id of the post
* PostTypeId (integer: 1 or 2) - 1 for questions, 2 for answers
* CreationDate (date) - post creation date in the format "YYYY-MM-DDTHH:MM:SS.ms"
* Tags (string, optional) - list of post tags, each tag is wrapped with html entities `&lt;` and `&gt;`
* OwnerUserId (integer, optional) - user id of the post's author
* ParentId (integer, optional) - for answers - id of the question
* Score (integer) - score (votes) of a question or an answer, can be negative (!)
* FavoriteCount (integer, optional) - how many times the question was added in the favorites

The second part of the dataset contains StackOverflow users.

The fields are the following and their order is also not defined:

* Id (integer) - user id
* Reputation (integer) - user's reputation
* CreationDate (string) - creation date in the format "YYYY-MM-DDTHH:MM:SS.ms"
* DisplayName (string) - user's name
* Location (string, options) - user's country
* Age (integer, optional) - user's age

## Step 3. Train your regexp skills

In this step you will find out how to parse information for complex rows! You will try to create some examples for parsing! There are some general rules for parsing:

1. To create a regular expression, which describes strings containing two patterns, where the order of the patterns is not defined use the following so-called ‘positive lookahead assertion’ with `?=` group modifier. For example, both strings “Washington Irving” and “Irving Washington” match the pattern:
```
(?=.*Washington)(?=.*Irving)
```.
2. To capture groups use round brackets. So, the pattern: `(?=.*(Washington))(?=.*(Irving))` captures `Washington` and `Irving` from both strings: "William Arthur Irving Washington was an English first-class Cricketer" and: “Washington Irving was an American writer”.
3. Use `\b` to specify boundaries of words and increase accuracy of your pattern. For example: pattern `(?=.*\bID=(\d+))(?=.*\bUserID=(\d+))` captures `1` and `2` from the string `ID=1 UserID=2`, whereas pattern without `\b`: `(?=.*ID=(\d+))(?=.*UserID=(\d+))` returns the wrong groups: `2` and `2`.
4. In Hive pattern for the external table in SERDEPROPERTIES `input.regex` should describe the whole input string, add `.*` at the end of the pattern.
5. Don't forget that for the beginning of string should also be covered. That's why use the pattern `.*?` for lazy initialization of future patterns.

To sum up, you can create your first regex for parsing Id from posts!

<b>Question!</b> What will be your first regex for parsing Id from the posts? Don't forget to add steps 4 and 5!

<div class="panel-group">
  <div class="panel panel-default">
    <div class="panel-heading">
      <h4 class="panel-title">
        <a data-toggle="collapse" href="#collapse-answer">Check your answer!</a>
      </h4>
    </div>
    <div id="collapse-answer" class="panel-collapse collapse">
      <div class="panel-body">The correct answer is `".*?(?=.*\\bId=\"(\\d+)\").*"` </div>
    </div>
  </div>
</div>

Let's create the first external table with one row which contains only `Id` field. Let's name it `posts_external_only_id`.
You can watch the lecture for the SerDe format: <a href="https://www.coursera.org/learn/big-data-analysis/lecture/wAGe6/hive-analytics-regexserde-views">Serde Format</a> and <a href="/notebooks/demos/course02_week02-Demo_submission.ipynb#2.-Creation-the-external-table">creation of external table</a> tutorial.

In [29]:
%%writefile demo_example.hql

-- adding necessary JARs and including database
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE demodb;
DROP TABLE IF EXISTS posts_external_only_id;


-- Create external table

-- Your code here
CREATE EXTERNAL TABLE posts_external_only_id 
(Id string) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.contrib.serde2.RegexSerDe'
WITH SERDEPROPERTIES ('input.regex'='.*?(?=.*\\bId=\"(\\d+)\").*') 
LOCATION '/data/stackexchange1000/posts'; 

Overwriting demo_example.hql


In [30]:
!hive -f demo_example.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
OK
Time taken: 1.111 seconds
OK
Time taken: 2.16 seconds
OK
Time taken: 0.58 seconds


Hooray! You have created your first table. Let us watch for this table!

In [21]:
%%writefile describe.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE demodb;
DESCRIBE posts_external_only_id;

Overwriting describe.hql


In [26]:
!hive -f describe.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 0.82 seconds
OK
id                  	string              	                    
Time taken: 1.133 seconds, Fetched: 1 row(s)


Let's see that the data is  correctly parsed. For this case, take a select query that chooses for us first 10 rows

In [35]:
%%writefile my_first_select.hql
-- Your code here
SELECT * FROM demodb.posts_external_only_id LIMIT 10;

Overwriting my_first_select.hql


In [36]:
!hive -f my_first_select.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
NULL
1394
3543
4521
8689
9062
14671
16307
18780
18929
Time taken: 2.754 seconds, Fetched: 10 row(s)


How many posts are there in the dataset? Don't forget to clear the `NULL` values!

In [37]:
%%writefile how_many_posts.hql
-- Your code here
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
select count(*) from demodb.posts_external_only_id where id is not null;

Overwriting how_many_posts.hql


In [38]:
!hive -f how_many_posts.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Query ID = jovyan_20190928182727_e5ab0644-3e0d-4922-b2e1-8f5c826bd4f9
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1569693762567_0002, Tracking URL = http://9eec40d2ff6f:8088/proxy/application_1569693762567_0002/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1569693762567_0002
Hadoop job information for Stage-1: number of mappers: 1; number of reduce

Try to parse different fields: for example, day and month of creation date. Don't forget that Hive will accept values for the capturing group in the lookahead. 

Now you are ready to complete your task! Before this you can check your regular expression for parsing!

In [33]:
import re

In [34]:
CHECK_ROW = '<row Id="1394" PostTypeId="2" ParentId="1390" CreationDate="2008-08-04T16:38:03.667" Score="16" Body="&lt;p&gt;Not sure how credible &lt;a href=&quot;http://www.builderau.com.au/program/windows/soa/Getting-started-with-Windows-Server-2008-Core-edition/0,339024644,339288700,00.htm&quot; rel=&quot;nofollow noreferrer&quot;&gt;this source is&lt;/a&gt;, but:&lt;/p&gt;&#xA;&#xA;&lt;blockquote&gt;&#xA;  &lt;p&gt;The Windows Server 2008 Core edition can:&lt;/p&gt;&#xA;  &#xA;  &lt;ul&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the file server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the Hyper-V virtualization server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the Directory Services role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the DHCP server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the IIS Web server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the DNS server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run Active Directory Lightweight Directory Services.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run the print server role.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;/ul&gt;&#xA;  &#xA;  &lt;p&gt;The Windows Server 2008 Core edition cannot:&lt;/p&gt;&#xA;  &#xA;  &lt;ul&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run a SQL Server.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run an Exchange Server.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run Internet Explorer.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run Windows Explorer.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Host a remote desktop session.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;li&gt;&lt;p&gt;Run MMC snap-in consoles locally.&lt;/p&gt;&lt;/li&gt;&#xA;  &lt;/ul&gt;&#xA;&lt;/blockquote&gt;&#xA;" OwnerUserId="91" LastEditorUserId="1" LastEditorDisplayName="Jeff Atwood" LastEditDate="2008-08-27T13:02:50.273" LastActivityDate="2008-08-27T13:02:50.273" CommentCount="1" />'

In [130]:
CHECK_REGEX = '^<row.*?(?=.*\\bId=\"(\\d+)\").*(?<=\\bCreationDate\\b=\")(\\d*)-(\\d*).*'
CHECK_REGEX = '.*(?<=\\bCreationDate\\b=\")(\\d*)-(\\d*)-(\\d*).*'

In [131]:
result = re.match(CHECK_REGEX, CHECK_ROW)

In [132]:
# Sanity check
assert result.group(0) == CHECK_ROW

In [133]:
# Check that your groups are correct
print(result.groups())

('2008', '08', '04')


## Step 4. Complete the assignment

In [1]:
%%writefile task1_create_external_table.hql
-- Create external table posts_sample_external with suitable values
-- Your code here
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;
#(?=.*\\bTags=\"(\\S+)\")?(?=.*\\bOwnerUserId=\"(\\d+)\")?(?=.*\\bParentId=\"(\\d+)\")?(?=.*\\bScore=\"(\\d+)\")(?=.*\\bFavoriteCount=\"(\\d+)\")?
USE demodb;
DROP TABLE IF EXISTS posts_sample_external;
CREATE EXTERNAL TABLE posts_sample_external 
(Id int, 
 year int,
 month int,
 PostTypeId int,
 Tags string,
 OwnerUserId int,
 ParentId int,
 Score int,
 FavoriteCount int
) 
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe' 
WITH SERDEPROPERTIES ('input.regex'='^<row.*(?=.*\\bId=\"(\\d+)\")(?=.*\\bPostTypeId=\"(\\d+)\").*(?<=\\bCreationDate\\b=\")(\\d*)-(\\d*).*') 
LOCATION '/data/stackexchange1000/posts';   

Overwriting task1_create_external_table.hql


In [2]:
!hive -f task1_create_external_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Exception in thread "main" java.lang.RuntimeException: org.apache.hadoop.hdfs.server.namenode.SafeModeException: Cannot create directory /tmp/mydir/jovyan/7ac67d31-e481-4e7e-82ed-ee26dd17d137. Name node is in safe mode.
The reported blocks 931 needs additional 2 blocks to reach the threshold 0.9990 of total blocks 934.
The number of live datanodes 1 has reached the minimum number 0. Safe mode will be turned off automatically once the thresholds have been reached.
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.newSafemodeException(FSNamesystem.java:1372)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.checkNameNodeSafeMode(FSNamesystem.java:1360)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.mkdirs(FSNamesystem.java:2969)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.mkdirs(NameNodeRpcServer.java:1078)
	at org.ap

Make sure that you have created your table correctly. Select the first 10 posts from the dataset.

In [160]:
%%writefile task1_check_select.hql

-- Write select query for the first 10 rows
-- Your code here
SELECT * FROM demodb.posts_sample_external LIMIT 10;

Overwriting task1_check_select.hql


In [153]:
!hive -f task1_check_select.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
NULL	NULL	NULL	NULL	NULL	NULL	NULL	NULL	NULL
1394	2	2008	8	NULL	91	NULL	16	NULL
3543	2	2008	8	NULL	399	NULL	37	NULL
4521	2	2008	8	NULL	358	NULL	25	NULL
8689	2	2008	8	NULL	1087	NULL	1	NULL
9062	2	2008	8	NULL	198	NULL	1	NULL
14671	2	2008	8	NULL	414	NULL	1	NULL
16307	2	2008	8	NULL	17	NULL	2	NULL
18780	2	2008	8	NULL	414	NULL	0	NULL
18929	2	2008	8	NULL	2031	NULL	13	NULL
Time taken: 2.236 seconds, Fetched: 10 row(s)


Create managed table `posts_sample`. Create the partition by the month and by the year. 

In [167]:
%%writefile task1_create_managed_table.hql
-- create managed table
-- Check that this table contains info about year and month
-- Your code here
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;

USE demodb;
DROP TABLE IF EXISTS posts_sample;
CREATE TABLE posts_sample 
(Id int, 
 PostTypeId int,
 Tags string,
 OwnerUserId int,
 ParentId int,
 Score int,
 FavoriteCount int
) 
PARTITIONED BY(year string,month string);   

Overwriting task1_create_managed_table.hql


In [171]:
!hive -f task1_create_managed_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
OK
Time taken: 0.8 seconds
OK
Time taken: 2.374 seconds
OK
Time taken: 0.86 seconds


Populate data from the table `posts_sample_external` to the table `posts_sample`. Don't forget about the partitioning rules!

In [175]:
%%writefile task1_insert_table.hql

-- Insert data to the managed table


USE demodb;
-- filling managed posts table from external one
SET hive.exec.dynamic.partition.mode=nonstrict;

-- Your code here for inserting data
set hive.exec.max.dynamic.partitions=1024;
set hive.exec.max.dynamic.partitions.pernode=512;
INSERT OVERWRITE TABLE POSTS_SAMPLE 
PARTITION (year,month)
SELECT Id,PostTypeId,year,month,Tags, OwnerUserId, ParentId, Score, FavoriteCount 
FROM posts_sample_external
WHERE YEAR IS NOT NULL and month IS NOT NULL;


Overwriting task1_insert_table.hql


In [176]:
!hive -f task1_insert_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 0.905 seconds
Query ID = jovyan_20190928212020_0812a975-8a24-46ad-83b7-180bb68c0ce3
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1569693762567_0007, Tracking URL = http://9eec40d2ff6f:8088/proxy/application_1569693762567_0007/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1569693762567_0007
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2019-09-28 21:20:49,162 Stage-1 map = 0%,  reduce = 0%
2019-09-28 21:21:39,272 Stage-1 map = 1%,  reduce = 0%, Cumulative CPU 37.12 sec
2019-09-28 21:22:39,467 Stage-1 map = 1%,  reduce = 0%, Cumulative CPU 90.79 sec
2019-09-28 21:22:58,289 Stage-1 map = 2%,  reduce = 0%, Cumulative CPU 107.4 sec
2019-09-28 21:23:40,182 Stage-1 map = 3%,  reduce = 0%, Cumulative CPU 145.49 sec
2019-09-2

Make sure that your table contains appropriate data about posts

In [179]:
%%writefile task1_watch_new_table.hql
-- Your code here
SELECT distinct year,month FROM demodb.posts_sample_external;

Overwriting task1_watch_new_table.hql


In [ ]:
!hive -f task1_watch_new_table.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Query ID = jovyan_20190928212828_ff9d45c0-d1d5-4d89-a322-c74af185da22
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1569693762567_0008, Tracking URL = http://9eec40d2ff6f:8088/proxy/application_1569693762567_0008/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1569693762567_0008
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2019-09-28 21:28:37,995 Stage-1 map = 0%,  reduce = 0%
2019-09-28 21:29:38,916 Stage-1 map = 0%,  reduce = 0%, Cumulative CPU 42

In [ ]:
%%writefile task1_watch_new_table.hql
-- Your code here
SELECT * FROM demodb.posts_sample LIMIT 10;

In [ ]:
!hive -f task1_watch_new_table.hql

Take the third row of the dataset in the ascending order for the posts (firstly by year, after that by month)

In [ ]:
%%writefile task1_result.hql
-- Your code here
use demodb;
select concat_ws("\t",year,m,line_count)
from
(select year, concat(year,"-",month) as m,count(*) as line_count,
 ROW_NUMBER() OVER (ORDER BY year,month) as row
from POSTS_SAMPLE
group by year,month
) as posts
where row=3;


In [ ]:
!hive -f task1_result.hql

## Step 5. Submission part. Do not touch!! And simple run all cells below!

Copy your notebook from the steps <a href="#Step-4.-Complete-the-assignment">Step 4</a> and <a href="#Step-5.-Submission-part.-Do-not-touch!!-And-simple-run-all-cells-below!">Step 5</a> to the new notebook. Run all the cells! And submit the copied notebook!

In [ ]:
!cat task1_create_external_table.hql > task1.hql
!cat task1_create_managed_table.hql >> task1.hql
!cat task1_insert_table.hql >> task1.hql
!cat task1_result.hql >> task1.hql

Take a look at your submission query!

In [ ]:
!cat task1.hql

In [ ]:
%%javascript

$(document).ready(function() {
    console.log('Ready');
    
    
    function is_hive_command(list_tokens) {
        return list_tokens.indexOf('hive') > -1 && 
             list_tokens.indexOf('f') > -1 &&
             list_tokens.indexOf('-') > -1 && 
             list_tokens.indexOf('!') > -1 &&
             list_tokens.indexOf('hql') > -1 && 
             list_tokens.indexOf('writefile') == -1;
    } 
    
    function collectText(input_tag) {

        var result_string = [];
        $.each($(input_tag).children(), function(index, child) {
            result_string.push($(child).text());
        });
        return [result_string, is_hive_command(result_string)];
    };
    
    var filtered_results = $(".cell.code_cell.rendered").filter(function(index, element) {
        var out = collectText($(element).find('.CodeMirror-line').find('span'));
        console.log(out);
        return collectText($(element).find('.CodeMirror-line').find('span'))[1];
    });
    $(filtered_results).remove();
});

In [ ]:
%%bash
hive -f task1.hql

Congratulations! You have completed the assignment! Now you can submit it to the system and get your results!

Copy your notebook from the steps <a href="#Step-4.-Complete-the-assignment">Step 4</a> and <a href="#Step-5.-Submission-part.-Do-not-touch!!-And-simple-run-all-cells-below!">Step 5</a> to the new notebook. Run all the cells! And submit the copied notebook!